In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from chatbot_test import ChatBotTs, load_api_key
from OktSentiment import Sentiment
from tarot import TarotBot

app = Flask(__name__)
CORS(app)

# ChatBot 인스턴스 생성
api_key = load_api_key()
if not api_key:
    raise ValueError("API key not found or invalid. Please check your api_key.txt file.")
chatbot = ChatBotTs(api_key, 'C:\\Users\\SMHRD\\Documents\\카카오톡 받은 파일\\유형,대처,유형설명(마카님 삭제).csv')  # CSV 파일 경로를 적절히 수정하세요
#C:\\Users\\SMHRD\\Documents\\카카오톡 받은 파일\\유형,대처,유형설명(마카님 삭제).csv
#c:/Users/SMHRD/원인-대처종합.csv

# Sentiment 인스턴스 생성
sentiment_analyzer = Sentiment()

@app.route("/chatbot", methods=["POST"])
def chat():
    data = request.json
    print("Received data:", data)
    user_nick = data.get("user_nick")
    user_input = data.get("input")
    
    if user_input:
        try:
            response = chatbot.process_input(user_nick, user_input)
            if response.get("isProblem"):
                # 형태소 분석 및 감정 분석 수행
                sentiment_results = sentiment_analyzer.parse([user_input])
                # 결과가 딕셔너리 형태로 바꾸고 소수점 소수점 둘쨰자리까지
                if 'polarity' in sentiment_results and len(sentiment_results['polarity']) > 0:
                    sentiment_results = sentiment_results['polarity'][0]
                    sentiment_results = {key: round(value * 100, 2) for key, value in sentiment_results.items()}

                response.update({
                    "sentiment": sentiment_results
                })

            return jsonify(response)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return jsonify({"error": f"An error occurred while processing your request: {str(e)}"}), 500
    else:
        return jsonify({"error": "No input provided"}), 400

# 타로카드 라우트
@app.route("/api/interpret",methods=["POST"])

def tarot_card():
    print(request.headers)
    print(request.data)
    
    data = request.get_json()
    cards = data.get("cards")
    user_mode = data.get("user_mode")
    user_input = data.get("user_input")
    user_select = data.get("user_select")
    print("recived data: ", data)
    if not cards or len(cards) != 3:
        return jsonify({"error": "카드를 다시 선택해 주세요."}), 400
            
    # key_path = "c:\\Users\\SMHRD\\api.txt"
    key_path = "C:\\Users\\SMHRD\\Desktop\\ky_api.txt"
    try:
        tarot = TarotBot(api_key_path=key_path)
        if user_mode == "일반":
            response = tarot.general_reading(user_select,cards)
            
        elif user_mode == "심리":
            print(user_input,cards)
            response = tarot.interpret_cards(user_input,cards)
        else:
            return jsonify({"error":"Invalid user_mode value."})
        return response
    except Exception as e:
        print("Error",str(e))
        return jsonify({"error":str(e)}), 500


if __name__ == "__main__":
    app.run(port=7000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [05/Sep/2024 17:11:00] "OPTIONS /api/interpret HTTP/1.1" 200 -


Host: localhost:7000
Connection: keep-alive
Content-Length: 149
Sec-Ch-Ua: "Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"
Content-Type: application/json
Sec-Ch-Ua-Mobile: ?0
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36
Sec-Ch-Ua-Platform: "Windows"
Accept: */*
Origin: http://localhost:3300
Sec-Fetch-Site: same-site
Sec-Fetch-Mode: cors
Sec-Fetch-Dest: empty
Referer: http://localhost:3300/
Accept-Encoding: gzip, deflate, br, zstd
Accept-Language: ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7


b'{"user_mode":"\xec\x8b\xac\xeb\xa6\xac","user_select":"\xec\x98\xa4\xeb\x8a\x98\xec\x9d\x98 \xec\x9a\xb4\xec\x84\xb8","cards":["Knight of Pentacles","Death","Six of Wands"],"user_input":"\xec\xa0\x80\xeb\x85\x81\xec\x97\x90 \xec\x9e\xa0\xec\x9d\x84 \xeb\xaa\xbb\xec\x9e\x90"}'
recived data:  {'user_mode': '심리', 'user_select': '오늘의 운세', 'cards': ['Knight of Pentacles', 'Death', 'Six of Wands'], 'user_input': '저

c:\Users\SMHRD\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
127.0.0.1 - - [05/Sep/2024 17:11:03] "POST /api/interpret HTTP/1.1" 200 -


response:  {'answer': {'cards': ['Knight of Pentacles', 'Death', 'Six of Wands'], 'text': '[\'Knight of Pentacles\', \'Death\', \'Six of Wands\']의 의미는 한 줄로 간략히 출력하면 다음과 같습니다:\n"꾸준한 노력과 변화, 성공과 승리의 조화"\n\n저녁에 잠을 못자시는 문제를 타로카드로 종합해 해석하면 다음과 같습니다:\nKnight of Pentacles는 꾸준한 노력과 인내를 상징하며, 현재 무언가에 열심히 몰두하고 있음을 나타냅니다. Death 카드는 큰 변화나 전환점을 의미하며, 현재의 상황이 끝나고 새로운 시작을 맞이해야 할 필요성을 시사합니다. Six of Wands는 성공과 승리를 상징하며, 앞으로의 긍정적인 결과를 예고합니다. 저녁에 잠을 못 자는 문제는 현재의 스트레스와 변화에 대한 적응이 필요하기 때문일 수 있습니다. 꾸준히 노력하고 적응함으로써 결국 성공적인 해결을 찾을 수 있을 것입니다.', 'user_input': '저녁에 잠을 못자'}}


127.0.0.1 - - [05/Sep/2024 17:11:48] "OPTIONS /api/interpret HTTP/1.1" 200 -


Host: localhost:7000
Connection: keep-alive
Content-Length: 149
Sec-Ch-Ua: "Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"
Content-Type: application/json
Sec-Ch-Ua-Mobile: ?0
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36
Sec-Ch-Ua-Platform: "Windows"
Accept: */*
Origin: http://localhost:3300
Sec-Fetch-Site: same-site
Sec-Fetch-Mode: cors
Sec-Fetch-Dest: empty
Referer: http://localhost:3300/
Accept-Encoding: gzip, deflate, br, zstd
Accept-Language: ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7


b'{"user_mode":"\xec\x9d\xbc\xeb\xb0\x98","user_select":"\xec\x98\xa4\xeb\x8a\x98\xec\x9d\x98 \xec\x9a\xb4\xec\x84\xb8","cards":["Knight of Pentacles","Death","Six of Wands"],"user_input":"\xec\xa0\x80\xeb\x85\x81\xec\x97\x90 \xec\x9e\xa0\xec\x9d\x84 \xeb\xaa\xbb\xec\x9e\x90"}'
recived data:  {'user_mode': '일반', 'user_select': '오늘의 운세', 'cards': ['Knight of Pentacles', 'Death', 'Six of Wands'], 'user_input': '저

127.0.0.1 - - [05/Sep/2024 17:11:51] "POST /api/interpret HTTP/1.1" 200 -


response:  {'answer': {'cards': 'Knight of Pentacles, Death, Six of Wands', 'text': '- Knight of Pentacles: 꾸준함과 신중함.\n- Death: 변화와 새로운 시작.\n- Six of Wands: 승리와 인정.\n\n오늘의 운세: 오늘은 꾸준하고 신중한 태도로 임하면 큰 변화를 맞이하게 될 것입니다. 이 변화는 처음에는 두려울 수 있지만, 결국 당신에게 긍정적인 결과를 가져다줄 것입니다. 변화에 적응하고 자신의 목표를 잊지 않는다면, 주변 사람들의 인정과 승리를 얻을 수 있을 것입니다. 일상 속에서 작은 성취들이 큰 성공으로 이어질 테니, 꾸준함과 인내심을 잃지 마세요.', 'user_select': '오늘의 운세'}}
